# Setup

Import a few common modules

In [1]:
#CODE for points = 1
# import sklearn, numpy, os

import sklearn 
import numpy as np
import os

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard



# to make this notebook's output stable across runs
np.random.seed(42)


# Vanishing/Exploding Gradients Problem

In [2]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [3]:
z = np.linspace(-5, 5, 200)

## Xavier and He Initialization

In [6]:
#CODE for points = 1
# set activation to relu and kernel initializer to he_normal

keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

### Leaky ReLU

In [18]:
#CODE for points = 1

# remember α is the hyperparameter that defines how much the function “leaks”
# set the value of alpha, use the value typically set

def leaky_relu(z, alpha=0.01):
    return np.maximum(alpha*z, z)

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [19]:
#CODE for points = 1
#load MNIST dataset from keras


(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data(path="mnist.npz")
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [20]:
#CODE for points = 1

# initialize kernel_initializer to "he_normal" and activation function to softmax

tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation='softmax')
])

In [21]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [22]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 7s 4ms/step - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980
Epoch 2/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596
Epoch 3/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834
Epoch 4/5
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940
Epoch 5/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006


Now look at what happens if we try to use the ReLU activation function instead:

In [23]:
np.random.seed(42)
tf.random.set_seed(42)

In [24]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))

model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))

for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

In [25]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [26]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 33s 17ms/step - loss: 1.9800 - accuracy: 0.2251 - val_loss: 1.6367 - val_accuracy: 0.3726
Epoch 2/5
1719/1719 [==============================] - 30s 17ms/step - loss: 1.5103 - accuracy: 0.3842 - val_loss: 1.1821 - val_accuracy: 0.5394
Epoch 3/5
1719/1719 [==============================] - 29s 17ms/step - loss: 1.1771 - accuracy: 0.5372 - val_loss: 0.9573 - val_accuracy: 0.6404
Epoch 4/5
1719/1719 [==============================] - 29s 17ms/step - loss: 1.0671 - accuracy: 0.5960 - val_loss: 0.9300 - val_accuracy: 0.6648
Epoch 5/5
1719/1719 [==============================] - 29s 17ms/step - loss: 0.8429 - accuracy: 0.7048 - val_loss: 0.7297 - val_accuracy: 0.7498


Not great at all, we suffered from the vanishing/exploding gradients problem.

# Batch Normalization

In [27]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [28]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_209 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_210 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [29]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [30]:
#bn1.updates #deprecated

In [31]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [32]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 12s 7ms/step - loss: 0.8619 - accuracy: 0.7335 - val_loss: 0.4687 - val_accuracy: 0.8716
Epoch 2/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.4346 - accuracy: 0.8734 - val_loss: 0.3494 - val_accuracy: 0.9038
Epoch 3/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3559 - accuracy: 0.8965 - val_loss: 0.3001 - val_accuracy: 0.9174
Epoch 4/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3115 - accuracy: 0.9101 - val_loss: 0.2695 - val_accuracy: 0.9242
Epoch 5/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.2815 - accuracy: 0.9175 - val_loss: 0.2444 - val_accuracy: 0.9306


Sometimes applying BN before the activation function works better (there's a debate on this topic). Moreover, the layer before a `BatchNormalization` layer does not need to have bias terms, since the `BatchNormalization` layer some as well, it would be a waste of parameters, so you can set `use_bias=False` when creating those layers:

In [33]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [34]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [35]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 12s 7ms/step - loss: 1.1288 - accuracy: 0.6813 - val_loss: 0.6171 - val_accuracy: 0.8486
Epoch 2/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.5778 - accuracy: 0.8548 - val_loss: 0.4354 - val_accuracy: 0.8920
Epoch 3/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.4590 - accuracy: 0.8818 - val_loss: 0.3585 - val_accuracy: 0.9108
Epoch 4/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3954 - accuracy: 0.8957 - val_loss: 0.3152 - val_accuracy: 0.9184
Epoch 5/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3552 - accuracy: 0.9033 - val_loss: 0.2840 - val_accuracy: 0.9252


# Faster Optimizers

## Momentum optimization

In [36]:
#CODE for points = 1
# initialize lr and momentum to typical values

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.0)

## Nesterov Accelerated Gradient

In [37]:
#CODE for points = 1
# initialize lr and momentum to typical values. Set nesterov so that it is used

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=True)

# Avoiding Overfitting Through Regularization

## $\ell_1$ and $\ell_2$ regularization

In [39]:
#CODE for points = 0.5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

In [41]:
#CODE for points = 1

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/
# nadam optimizer

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer='nadam', metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 16s 9ms/step - loss: 1.2773 - accuracy: 0.8766 - val_loss: 0.8154 - val_accuracy: 0.8884
Epoch 2/2
1719/1719 [==============================] - 15s 8ms/step - loss: 0.8188 - accuracy: 0.8850 - val_loss: 0.7758 - val_accuracy: 0.9030


In [42]:
#CODE for points = .5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

In [44]:
#CODE for points = 1

# activation function initialized as softmax
# nadam optimizer


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation='softmax')
])
model.compile(loss="sparse_categorical_crossentropy", optimizer='nadam', metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))


Epoch 1/2
1719/1719 [==============================] - 15s 8ms/step - loss: 1.2926 - accuracy: 0.8764 - val_loss: 0.8167 - val_accuracy: 0.8898
Epoch 2/2
1719/1719 [==============================] - 14s 8ms/step - loss: 0.8209 - accuracy: 0.8851 - val_loss: 0.7753 - val_accuracy: 0.9026
